In [ ]:
# كود التجهيز الأولي لربط جوجل درايف ورفع ملفات API
# -*- coding: utf-8 -*-
# استيراد مكتبة جوجل درايف
from google.colab import drive
# استيراد مكتبة التعامل مع الملفات في جوجل كولاب
from google.colab import files

# تثبيت أي مكتبات ضرورية
!pip install kaggle

# طلب ربط الحساب وتحديد نقطة الوصول
drive.mount('/content/drive')


# --- رفع ملفات API ---

# 1. رفع ملف Kaggle API
print('Please upload the "kaggle.json" file:')
files.upload()

# 2. رفع ملف Unsplash API (تمت إضافة هذا الجزء)
print('\nPlease upload the "Unsplash.json" file:')
files.upload()


# --- إعداد Kaggle API ---

# إنشاء مجلد مخفي باسم .kaggle في الدليل الرئيسي
!mkdir -p ~/.kaggle

# نسخ ملف كاجل الذي تم رفعه إلى المجلد الجديد
!cp kaggle.json ~/.kaggle/

# تعيين أذونات القراءة والكتابة للمستخدم فقط
!chmod 600 ~/.kaggle/kaggle.json

print("\n--- Setup Complete. Both Kaggle and Unsplash API files are ready. ---")

In [ ]:
#  تحضير بيانات صور للكلاب والقطط من كاجل للاختبار

# -*- coding: utf-8 -*-
import kagglehub
import os

# --- الخطوة 1: تحميل مسار مجموعة البيانات ---
print("--- Downloading path for the new dataset: 'tongpython/cat-and-dog' ---")
new_dataset_path = kagglehub.dataset_download("tongpython/cat-and-dog")
print(f"\nDataset path: {new_dataset_path}")

# --- الخطوة 2: استكشاف هيكل المجلدات والملفات ---
# نستخدم الأمر ls -R لعرض كل المحتويات بشكل شجري
# قد يستغرق هذا الأمر بعض الوقت لأن البيانات كثيرة
print("\n--- Exploring the directory structure... ---")
!ls -R {new_dataset_path}

In [ ]:
# الاختبار على بيانات صور للكلاب والقطط من كاجل

# -*- coding: utf-8 -*-
import tensorflow as tf
import os
import numpy as np
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# =================================================================
# الخطوة 1: تحميل النموذج وتحديد المسارات
# =================================================================
print("--- Loading the best trained model... ---")
best_model_path = "/content/drive/MyDrive/ML_Cat_Dog_Project/checkpoints/best_model.keras"
model = tf.keras.models.load_model(best_model_path)
print("Model loaded successfully.")

path_to_training_data = os.path.join(new_dataset_path, 'training_set', 'training_set')
path_to_test_data = os.path.join(new_dataset_path, 'test_set', 'test_set')

# =================================================================
# الخطوة 2: جمع مسارات الصور وتسمياتها الحقيقية
# =================================================================
all_image_paths = []
all_image_labels = []
class_names = ['cats', 'dogs']

def collect_paths_and_labels(data_dir, label_name):
    label_index = class_names.index(label_name)
    class_dir = os.path.join(data_dir, label_name)
    if os.path.exists(class_dir):
        for f in os.listdir(class_dir):
            all_image_paths.append(os.path.join(class_dir, f))
            all_image_labels.append(label_index)

for data_dir in [path_to_training_data, path_to_test_data]:
    if os.path.exists(data_dir):
        collect_paths_and_labels(data_dir, 'cats')
        collect_paths_and_labels(data_dir, 'dogs')
print(f"\nFound a total of {len(all_image_paths)} file paths to process.")


# =================================================================
# الخطوة 3: بناء خط إمداد بيانات قوي
# =================================================================
IMG_SIZE = (180, 180)
BATCH_SIZE = 64

def load_and_decode_image(filepath, label):
    try:
        img_bytes = tf.io.read_file(filepath)
        img = tf.io.decode_image(img_bytes, channels=3, expand_animations=False)
        img = tf.image.resize(img, IMG_SIZE)
        return img, label, True
    except:
        return tf.zeros((IMG_SIZE[0], IMG_SIZE[1], 3)), tf.constant(-1, dtype=tf.int32), False

full_ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))
processed_ds = full_ds.map(lambda fp, lbl: tf.py_function(load_and_decode_image, [fp, lbl], [tf.float32, tf.int32, tf.bool]))
valid_ds = processed_ds.filter(lambda img, lbl, success: success)

# --- تم تصحيح هذا الجزء ---
# استخراج التسميات الحقيقية باستخدام np.array بدلاً من np.concatenate
y_true = np.array([y for x, y, z in valid_ds])
images_only_ds = valid_ds.map(lambda x, y, z: x)

batched_ds = images_only_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# =================================================================
# الخطوة 4: تشغيل التنبؤات وحساب المقاييس
# =================================================================
print("\n--- Starting prediction on all valid images... ---")
start_time = time.time()
y_pred_probs = model.predict(batched_ds, verbose=1)
end_time = time.time()
print(f"Prediction finished in {end_time - start_time:.2f} seconds.")

y_pred_classes = (y_pred_probs > 0.5).astype(int).flatten()

print("\n--- Calculating final metrics... ---")
accuracy = accuracy_score(y_true, y_pred_classes)
cm = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes, target_names=class_names, zero_division=0)

# =================================================================
# الخطوة 5: بناء وحفظ التقرير النهائي
# =================================================================
timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
report_string = f"""
=================================================
      FULL PERFORMANCE REPORT (LARGE DATASET)
=================================================
Test Time: {timestamp}
Dataset Source: tongpython/cat-and-dog
Total Images Processed: {len(y_true)}
-------------------------------------------------

### OVERALL ACCURACY ###
Accuracy: {accuracy:.2%}

-------------------------------------------------

### CLASSIFICATION REPORT ###
{class_report}

-------------------------------------------------

### TEXT-BASED CONFUSION MATRIX ###

                 Predicted Cat | Predicted Dog
               ----------------|----------------
    Actual Cat |      {cm[0, 0]:<7} |      {cm[0, 1]:<7}
    Actual Dog |      {cm[1, 0]:<7} |      {cm[1, 1]:<7}

-------------------------------------------------
Matrix Details:
- True Cats (TN): {cm[0, 0]}
- False Dogs (FP): {cm[0, 1]}  (Cat predicted as Dog)
- False Cats (FN): {cm[1, 0]}  (Dog predicted as Cat)
- True Dogs (TP): {cm[1, 1]}
=================================================
"""

report_dir = "/content/drive/MyDrive/ML_Cat_Dog_Project/test_reports"
os.makedirs(report_dir, exist_ok=True)
report_filename = f"large_dataset_report_{time.strftime('%Y%m%d-%H%M%S')}.txt"
report_path = os.path.join(report_dir, report_filename)
with open(report_path, 'w') as f:
    f.write(report_string)

print(f"\n---> Detailed performance report saved to: {report_path} <---")
print(report_string)

In [ ]:
# تجربة جلب صور من الانترنت من موقع Unsplash عبر API عدد الصور 60 صورة
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
import os
import time
import requests
import json
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# =================================================================
# الخطوة 1: تحميل الإعدادات (لا تغيير هنا)
# =================================================================
print("--- Loading required assets (model and API keys)... ---")
best_model_path = "/content/drive/MyDrive/ML_Cat_Dog_Project/checkpoints/best_model.keras"
model = tf.keras.models.load_model(best_model_path)
print("Best model loaded successfully.")

with open('Unsplash.json') as f:
    secrets = json.load(f)
    unsplash_access_key = secrets['unsplash_access_key']
print("Unsplash API key loaded successfully.")

# =================================================================
# الخطوة 2: تعريف الدوال الأساسية (مع تبسيط دالة الجلب)
# =================================================================
IMG_SIZE = (180, 180)
HEADERS = {'Authorization': f'Client-ID {unsplash_access_key}'}
class_names = ['cats', 'dogs']

# --- تم تبسيط هذه الدالة ---
def fetch_and_predict(search_query, true_label_index, num_images_to_get):
    """
    دالة تقوم بطلب واحد لجلب دفعة من الصور وتصنيفها.
    """
    print(f"\n--- Starting process for query: '{search_query}' ---")
    y_true_class, y_pred_class = [], []

    # سنقوم بطلب واحد فقط مع تحديد العدد (الحد الأقصى 30)
    api_url = f"https://api.unsplash.com/search/photos?query={search_query}&per_page={num_images_to_get}"

    try:
        response = requests.get(api_url, headers=HEADERS)
        response.raise_for_status()
        data = response.json()

        for item in data['results']:
            image_url = item['urls']['regular']
            try:
                image_response = requests.get(image_url, stream=True)
                image_response.raise_for_status()

                img_bytes = tf.constant(image_response.content)
                img = tf.io.decode_image(img_bytes, channels=3, expand_animations=False)
                img = tf.image.resize(img, IMG_SIZE)
                img_array = tf.expand_dims(img, 0)

                prediction = model.predict(img_array, verbose=0)
                predicted_class_index = 1 if prediction[0][0] > 0.5 else 0

                y_true_class.append(true_label_index)
                y_pred_class.append(predicted_class_index)
            except Exception:
                continue # تجاهل أي صورة تفشل في التحميل

    except requests.exceptions.RequestException as e:
        print(f"API request failed. Error: {e}")

    print(f"Successfully processed {len(y_true_class)} images for '{search_query}'.")
    return y_true_class, y_pred_class

# =================================================================
# الخطوة 3: تنفيذ الاختبار وجمع النتائج
# =================================================================
# --- تم تعديل العدد هنا إلى 30 ---
NUM_IMAGES_PER_CLASS = 30

y_true_cats, y_pred_cats = fetch_and_predict('cat', 0, NUM_IMAGES_PER_CLASS)
y_true_dogs, y_pred_dogs = fetch_and_predict('dog', 1, NUM_IMAGES_PER_CLASS)

y_true_final = y_true_cats + y_true_dogs
y_pred_final = y_pred_cats + y_pred_dogs

# =================================================================
# الخطوة 4: حساب المقاييس وإنشاء التقرير (لا تغيير هنا)
# =================================================================
print("\n--- All images processed. Calculating final metrics... ---")
if not y_true_final:
    print("No images were successfully processed. Cannot generate report.")
else:
    # ... (بقية كود إنشاء التقرير وحفظه كما هو) ...
    accuracy = accuracy_score(y_true_final, y_pred_final)
    cm = confusion_matrix(y_true_final, y_pred_final)
    class_report = classification_report(y_true_final, y_pred_final, target_names=class_names, zero_division=0)

    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    report_string = f"""
=================================================
      UNSPLASH API TEST - PERFORMANCE REPORT
=================================================
Test Time: {timestamp}
Target Images Per Class: {NUM_IMAGES_PER_CLASS}
Total Images Processed: {len(y_true_final)}
-------------------------------------------------
### OVERALL ACCURACY ###
Accuracy: {accuracy:.2%}
-------------------------------------------------
### CLASSIFICATION REPORT ###
{class_report}
-------------------------------------------------
### TEXT-BASED CONFUSION MATRIX ###
                 Predicted Cat | Predicted Dog
               ----------------|----------------
    Actual Cat |      {cm[0, 0]:<7} |      {cm[0, 1]:<7}
    Actual Dog |      {cm[1, 0]:<7} |      {cm[1, 1]:<7}
-------------------------------------------------
Matrix Details:
- True Cats (TN): {cm[0, 0]}
- False Dogs (FP): {cm[0, 1]}  (Cat predicted as Dog)
- False Cats (FN): {cm[1, 0]}  (Dog predicted as Cat)
- True Dogs (TP): {cm[1, 1]}
=================================================
"""
    report_dir = "/content/drive/MyDrive/ML_Cat_Dog_Project/test_reports"
    os.makedirs(report_dir, exist_ok=True)
    report_filename = f"unsplash_report_{time.strftime('%Y%m%d-%H%M%S')}.txt"
    report_path = os.path.join(report_dir, report_filename)
    with open(report_path, 'w') as f:
        f.write(report_string)
    print(f"\n---> Detailed performance report saved to: {report_path} <---")
    print(report_string)